In [1]:
#!/usr/bin/env python3
# -*- coding : utf-8 -*-
#
##
### author: zhzhang
### e-mail: zhzhang2015@sina.com / zhenhua.zhang217@gmail.com
### date  : 2018.10.25
##
#
################################################################################

print(__doc__)

# Load basic modules
import os
import sys                                                                      
import time
import logging

from os.path import join

import warnings
warnings.filterwarnings('ignore')

ct = time.clock()  # Time counting starts

# Create stream handler of logging
## Logging info formatter
FORMATTER = '%(asctime)s <%(name)s> %(levelname)s: %(message)s'
formatter = logging.Formatter(FORMATTER, '%Y-%m-%d,%H:%M:%S')

## Set up main logging stream and formatter
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)

# Set up logging
lg = logging.getLogger()
lg.setLevel(logging.INFO)         # default logging level INFO
lg.addHandler(ch)
lg.info("=== Start ... ===")


2018-11-09,09:34:05 <root> INFO: === Start ... ===


Automatically created module for IPython interactive environment


In [2]:
# Load necessay modules
lg.info("Load necessay modules...")

import numpy as np
lg.info('Loaded library: %s. Version: %s'%('numpy', np.__version__))

import sklearn
lg.info('Loaded library: %s. Version: %s'%('scikit-learn', sklearn.__version__))

import pandas as pd
lg.info('Loaded library: %s. Version: %s'%('pandas', pd.__version__))


# Modules from scikit-learn

# Load functions for model selection
from sklearn.model_selection import cross_validate

2018-11-09,09:34:06 <root> INFO: Load necessay modules...
2018-11-09,09:34:06 <root> INFO: Loaded library: numpy. Version: 1.15.1
2018-11-09,09:34:06 <root> INFO: Loaded library: scikit-learn. Version: 0.19.2
2018-11-09,09:34:07 <root> INFO: Loaded library: pandas. Version: 0.23.4


In [4]:
def plot_learning_curve(lc, title, ylim=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    lc : tuple, including train_sizes_abs, train_scores, test_scores
         train_sizes_abs : array, shape (n_unique_ticks,), dtype int
        Numbers of training examples that has been used to generate the learning curve. Note that the number of ticks might be less than n_ticks because duplicate entries will be removed.

         train_scores : array, shape (n_ticks, n_cv_folds)
        Scores on training sets.

         test_scores : array, shape (n_ticks, n_cv_folds)
        Scores on test set.

    title : string
        Title for the chart.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = lc
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [5]:
# Arrange working dirs and files
ipfd = '/home/umcg-zzhang/Documents/projects/ASEpredictor/outputs/biosGavinOverlapCov10'
ipfn = 'biosGavinOverlapCov10Anno.tsv'
ipf = join(ipfd, ipfn)

# Loading training datasets. 
ipdf = pd.read_table(ipf, header=0)

In [6]:
# Implement a function to select interested features
def featureSelect():
    pass

In [7]:
# Load necessary methods
# from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split

# Setting up random seed
np.random.seed(1234)

# Predictors v.s. responsible variables
X_c, y_c = ['chr', 'pos', 'ref', 'alt', 'cadd', 'group'], ['log2FC']

X, y = ipdf.loc[:, X_c], ipdf.loc[:, y_c]

## Encoding categorical feature?
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(X['ref'].append(X['alt']))
X['refEncoded'] = le.transform(X['ref'])
X['altEncoded'] = le.transform(X['alt'])
X['groupEncoded'] = le.fit_transform(X['group'])

XData_c = ['chr', 'pos', 'refEncoded', 'altEncoded', 'cadd', 'groupEncoded']
X_data = X.loc[:, XData_c]
# Split of training datasets. 
X_train, X_test, y_train, y_test =  train_test_split(X_data, y, random_state=42)

In [8]:
print("X_train.shape: ", X_train.shape)
print("X_test.shape: ", X_test.shape)
print("y_train.shape: ", y_train.shape)
print("y_test.shape: ", y_test.shape)

X_train.shape:  (7499, 6)
X_test.shape:  (2500, 6)
y_train.shape:  (7499, 1)
y_test.shape:  (2500, 1)


In [9]:
# Preprocessing or transformers

## Imputation of missing values?
## from sklearn.impute import MissingIndicator
## mi = MissingIndicator()

## Since we have some outliers, the RobustScaler will be used.
from sklearn.preprocessing import RobustScaler
rs = RobustScaler(quantile_range=(25,75))

## Normalizatoin?
from sklearn.preprocessing import Normalizer
nm = Normalizer()

In [10]:
#
##
### The best parameter values should ALWAYS be CROSS-VALIDATED.
### Seting up `seed` by `numpy.random.seed()`, since scikit-learn used `numpy.random()` throughout.
### A trick to speed up is to use `memory` parameter to cache the estimators
##
#

# Load learning algorithm
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=10)

/home/umcg-zzhang/Tools/Anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [11]:
# Aggregate several steps into a pipeline by Pipeline
from sklearn.pipeline import Pipeline

rfr_estimators = [
#    ('mi', mi),    # MissingIndicator
#    ('ohe', ohe),  # OneHotEncoder
    ('rs', rs),     # RobustScaler
    ('nm', nm),     # Normalizer
    ('rfr', rfr),   # RandomForestClassifier
]

rfr_pipeline = Pipeline(rfr_estimators)
rfrpf = rfr_pipeline.fit(X_data, y)
rfrm = rfrpf.named_steps['rfr']
rfrm.feature_importances_

array([0.37410995, 0.08123148, 0.18284564, 0.21936131, 0.13502532,
       0.0074263 ])

In [12]:
# Cross validation metohd 1
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

# Using k-fold CV (plain CV) or other strategies
plainCV = False

if plainCV: 
    cv = 10
else:
    cv = ShuffleSplit(
        n_splits=10, test_size=0.3, random_state=0
    )  # optimization need
cvs = cross_val_score(rfr_pipeline, X_data, y, cv=cv)
lc = learning_curve(rfr_pipeline, X_data, y, train_sizes=np.linspace(0.1, 1.0, 10), cv=cv, n_jobs=5)
plot_learning_curve(lc, '7K sample test').show()

<Figure size 640x480 with 1 Axes>

In [13]:
# Cross validation method 2 
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
skf = KFold(n_splits=10)
for train, test in skf.split(X_data, y):
    rfrpf = rfr_pipeline.fit(X_data.loc[train, ], y.loc[train, ])
    rfrpfp = rfrpf.predict(X_data.loc[test, ])
    print(explained_variance_score(y.loc[test, ], rfrpfp))

0.6658521520582068
0.7173385463533447
0.7591956978029426
0.6292456069384744
0.8148944258420552
0.7171855945369954
0.7437437330337315
0.6261965503535979
0.6670213772765498
-0.029200129267857866


In [ ]:
# Model evalution
# Load functions for model evaluation (classification)
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

In [ ]:
# Modules to of plots
import matplotlib as mpl
from matplotlib import pyplot as plt